<h3 align="center"><FONT size="7pt">ARTHENA - Parsing Challenge</FONT></h3>
<br><h3 align="center"><FONT size="5pt">Pierre-Charles Paret-Van Wolput</FONT></h3></br>

### Import part

In [1]:
import os
from os import listdir
from html.parser import HTMLParser
import json
import sys
import numpy as np

In [2]:
path1 = "./data/2015-03-18/"

In [3]:
path2 = "./data/2017-12-20/"

### Parser class modification for first filetype

I used an HTML parser whose methods I can override to produce expected results. It could also be done by starting the whole string parsing if needed.

This parser is case specific.

In [4]:
class MyHTMLParserType1(HTMLParser):
    
    #Variables to know when we are treating specific strings
    is_name = False
    is_title = False
    is_price = False
    next_div_is_price = False #warns that the next div tag encountered gives the price
    count_h3 = 0 #counts the number of h3 tags encountered
    
    #Specific variables during a single file processing
    name = ""
    title = ""
    currency = ""
    amount = ""
    
    #Variables for storage
    dict_artists_works = {}
    
    #Resets the parser entirely
    def reset_attributes(self):
        self.reset_infile_variables()
        self.dict_artists_works = {}    
    
    #Getter for output
    def get_artists_works_array_of_dicts(self):
        dict_artists_works = self.dict_artists_works
        artists = list(dict_artists_works) #access the keys of the dict
        array_dicts = [{} for x in range(len(artists))]
        
        #fill the array of new dicts 
        for i in range(len(artists)):
            artist = artists[i]
            dict = {"artist": artist, "works": dict_artists_works[artist]}
            array_dicts[i] = dict
        
        return(array_dicts)
    
    #Adds a artwork to the dicts
    def add_artwork(self):
        name = self.name
        #I used an dict to define the artwork
        artwork = {"title": self.title, "currency": self.currency, "amount": self.amount}
        if name in self.dict_artists_works:
            self.dict_artists_works[name].append(artwork)
        else:
            self.dict_artists_works[name] = [artwork]
           
    
    #(Re)setter for variables used to locate informations (name, etc.)
    def reset_infile_variables(self):
        self.is_name = False
        self.is_title = False
        self.name = ""
        self.title = ""
        self.currency = ""
        self.amount = 0
        self.count_h3 = 0 
        
    
    #Method treating beginning of tags to know what to do with the data 
    def handle_starttag(self, tag, attrs):
        #reset the variables that are file-related
        if tag == 'html':
            self.reset_infile_variables()
        #warn that next data to treat is name
        if tag == 'h2':
            self.is_name = True
        #warn that next data to treat is title
        if tag == 'h3':
            if self.count_h3 == 0:
                self.is_title = True
            if self.count_h3 == 1:
                self.next_div_is_price = True
            self.count_h3 += 1
        #warn that next data to treat is price
        if (tag == 'div') & self.next_div_is_price:
            self.is_price = True
            self.next_div_is_price = False
    
    #Method treating beginning of tags to stop the special treatment of the data
    def handle_endtag(self, tag):
        #reset the variables that are file-related and add new object
        if tag == 'html':
            self.add_artwork()
            self.reset_infile_variables()
        #warn that we are done treating name
        if tag == 'h2':
            self.is_name = False
        #warn that we are done treating title
        if tag == 'h3':
            self.is_title = False
        #warn that we are done treating price
        if tag == 'div':
            self.is_price = False

    def handle_data(self, data):
        #name treatement
        if self.is_name:
            string_components = data.split(' ')
            string_name = "" 
            for s in string_components:
                if (s[0].isalpha()):
                    string_name = string_name + s + ' '
            string_name = string_name[0:len(string_name)-1] #remove space after end of last name
            self.name = string_name
        #title treatment
        if self.is_title:
            self.title = data
        #price treatment
        if self.is_price:
            string_components = data.split(' ')
            string_amount = string_components[1]
            if (string_amount.find(',') != -1):
                string_amount = string_amount.replace(',','') #replace the , of the thousands
            self.currency = string_components[0]
            self.amount = float(string_amount)

### Main for 1 directory

In [19]:
def main_parsing_HTML_to_JSON_1dir(path_directory):
    
    filenames = os.listdir(path_directory)
    if (path_directory == './data/2015-03-18/'):
        parser = MyHTMLParserType1()
        parser.reset_attributes()
    else:
        if (path_directory == './data/2017-12-20/'):
            parser = MyHTMLParserType2()
            parser.reset_attributes()
    
    encoder = json.JSONEncoder()
    
    #Parser feeding
    for filename in filenames:
        
        #File Opening
        f = open(path_directory + filename, 'r')
        string_file = f.read()
        f.close()
        
        print('Opening finished for file' + filename)
        
        parser.feed(string_file)
    
    #Preparation of outputs of the function
    array_artists_dicts = parser.get_artists_works_array_of_dicts()
    parser.reset_attributes()
    parser.close()
    array_artists_json = encoder.encode(array_artists_dicts)
        
    return(array_artists_json)

### Treatment of first directory

In [20]:
out1 = main_parsing_HTML_to_JSON_1dir(path1)

Opening finished for filelot1.html
Opening finished for filelot5.html
Opening finished for filelot4.html
Opening finished for filelot3.html
Opening finished for filelot2.html


In [22]:
sys.stdout.write(out1)

[{"artist": "Rembrandt Harmensz. van Rijn", "works": [{"title": "Christ at Emmaus: The smaller Plate", "currency": "GBP", "amount": 6875.0}]}, {"artist": "Marc Chagall", "works": [{"title": "Self-Portrait", "currency": "GBP", "amount": 6000.0}]}, {"artist": "Pablo Picasso", "works": [{"title": "Quatre Femmes nues et T\u00eate sculpt\u00e9e, from: La Suite Vollard", "currency": "USD", "amount": 25000.0}]}, {"artist": "Joan Mir\u00f3", "works": [{"title": "Femme et Chien devant la Lune", "currency": "GBP", "amount": 11250.0}]}, {"artist": "Louis Marcoussis", "works": [{"title": "La Table", "currency": "GBP", "amount": 9200.0}]}]

### Parser class modification for first filetype

In [9]:
class MyHTMLParserType2(HTMLParser):
    
    #Variables to know when we are treating specific strings
    is_name = False
    is_title = False
    is_currency = False
    is_amount = False
    count_h3 = 0 #counts the number of h3 tags encountered
    
    #Specific variables during a single file processing
    name = ""
    title = ""
    currency = ""
    amount = ""
    
    #Variables for storage
    dict_artists_works = {}
    
    def reset_attributes(self):
        self.reset_infile_variables()
        self.dict_artists_works = {}
    
    #Getter for output
    def get_artists_works_array_of_dicts(self):
        dict_artists_works = self.dict_artists_works
        artists = list(dict_artists_works) #access the keys of the dict
        array_dicts = [{} for x in range(len(artists))]
        
        #fill the array of new dicts 
        for i in range(len(artists)):
            artist = artists[i]
            dict = {"artist": artist, "works": dict_artists_works[artist]}
            array_dicts[i] = dict
        
        return(array_dicts)
    
    #Adds a artwork to the dicts
    def add_artwork(self):
        name = self.name
        #I used an dict to define the artwork
        artwork = {"title": self.title, "currency": self.currency, "amount": self.amount}
        if name in self.dict_artists_works:
            self.dict_artists_works[name].append(artwork)
        else:
            self.dict_artists_works[name] = [artwork]
           
    
    #(Re)setter for variables used to locate informations (name, etc.)
    def reset_infile_variables(self):
        self.is_name = False
        self.is_title = False
        self.name = ""
        self.title = ""
        self.currency = ""
        self.amount = 0
        self.count_h3 = 0 
        
    
    #Method treating beginning of tags to know what to do with the data 
    def handle_starttag(self, tag, attrs):
        attrs_empty = (len(attrs) ==0)
        #reset the variables that are file-related
        if tag == 'html':
            self.reset_infile_variables()
        #warn that next data to treat is name
        if (tag == 'h3') & (not attrs_empty):
            self.is_name = True
        #warn that next data to treat is title
        if (tag == 'h3') & (attrs_empty):
            if self.count_h3 == 0:
                self.is_title = True
            self.count_h3 += 1
        #warn that next data to treat is currency
        if (tag == 'span') & (not attrs_empty):
            self.is_currency = True
        #warn that next data to treat is amount
        if (tag == 'span') & (attrs_empty):
            self.is_amount = True
    
    #Method treating beginning of tags to stop the special treatment of the data
    def handle_endtag(self, tag):
        #reset the variables that are file-related and add new object
        if tag == 'html':
            self.add_artwork()
            self.reset_infile_variables()
        #warn that we are done treating name or title
        if tag == 'h3':
            self.is_name = False
            self.is_title = False
        #warn that we are done treating price
        if tag == 'span':
            self.is_currency = False
            self.is_amount = False

    def handle_data(self, data):
        #name treatement
        if self.is_name:
            string_components = data.split(' ')
            string_name = "" 
            for s in string_components:
                if (s[0].isalpha()):
                    string_name = string_name + s + ' '
            string_name = string_name[0:len(string_name)-1] #remove space after end of last name
            self.name = string_name
        #title treatment
        if self.is_title:
            self.title = data
        #currency treatment
        if self.is_currency:
            self.currency = data
        #amount treatment
        if self.is_amount:
            string_amount = data
            if (string_amount.find(',') != -1):
                string_amount = string_amount.replace(',','') #replace the , of the thousands
            self.amount = float(string_amount)

### Treatment of second directory

In [10]:
out2 = main_parsing_HTML_to_JSON_1dir(path2)

Opening finished for filelot1.html
Opening finished for filelot5.html
Opening finished for filelot4.html
Opening finished for filelot3.html
Opening finished for filelot2.html


In [11]:
sys.stdout.write(out2)

[{"artist": "Rembrandt Harmensz. van Rijn", "works": [{"title": "Self-Portrait in a Cap and Scarf with Face dark: Bust", "currency": "USD", "amount": 8125.0}]}, {"artist": "Pablo Picasso", "works": [{"title": "Quatre Femmes nues et T\u00eate sculpt\u00e9e, from: La Suite Vollard", "currency": "USD", "amount": 36000.0}, {"title": "Femme assise, robe bleue", "currency": "USD", "amount": 45047500.0}]}, {"artist": "Louis Marcoussis", "works": [{"title": "Bar du porto", "currency": "USD", "amount": 149000.0}, {"title": "Portrait d'Alice Halicka", "currency": "USD", "amount": 182500.0}]}]

### Main for full file

Note: doing this in Python was maybe a bad idea due to the type of objects needed.

In [28]:
def main_treatment_HTML_to_JSON(path_upper_directory):
    dirnames = os.listdir(path_upper_directory)
    
    #Storage variable to merge the two results of the datasets
    storage = []
    
    for dirname in dirnames:
        path_directory = path_upper_directory + "/" + dirname + "/"
        
        filenames = os.listdir(path_directory)
        
        #Choice of a different parser according to the type of the file
        if (path_directory == './data/2015-03-18/'):
            parser = MyHTMLParserType1()
            parser.reset_attributes()
        else:
            if (path_directory == './data/2017-12-20/'):
                parser = MyHTMLParserType2()
                parser.reset_attributes()

        #Parser feeding
        for filename in filenames:

            #File Opening
            f = open(path_directory + filename, 'r')
            string_file = f.read()
            f.close()

            print('Opening finished for file' + filename)

            parser.feed(string_file)
        storage.append(parser.get_artists_works_array_of_dicts())
        parser.reset_attributes()
        parser.close()
    
    #Creation of the merged array
    first_array_artists = storage[0]
    second_array_artists = storage[1]
    
    array_artists_dicts = []
    
    #Storage of the names of the artists in the first array
    artists_names_first_array = [dict_artist["artist"] for dict_artist in first_array_artists]
    
    #Filling of the merged array with python dicts for the artists present in first set or in the two sets
    for dict_artist_1  in first_array_artists:        
        for dict_artist_2 in second_array_artists:
            #If artist in already present, add the works to his name
            if dict_artist_1["artist"] == dict_artist_2["artist"]:
                for work in dict_artist_2["works"]:
                    dict_artist_1["works"].append(work)
                array_artists_dicts.append(dict_artist_1)
    
    #Filling of the merged array with python dicts for the artists present in the second set only
    for dict_artist in second_array_artists:
        artist = dict_artist["artist"]
        is_new = True
        #If the artist is not in the first array, add his full dict to the merged array
        for name in artists_names_first_array:
            if artist == name:
                isNew = False
        if is_new:
           array_artists_dicts.append(dict_artist) 
    #Money conversion
    for i in range(len(array_artists_dicts)):
        dict_artist = array_artists_dicts[i]
        array_works = dict_artist["works"]
        for j in range(len(array_works)):
            work = array_works[j]
            if work["currency"] == "GBP":
                work["currency"] = "USD"
                work["amount"] *= 1.34
            array_works[j] = work
        dict_artist["works"] = array_works 
        array_artists_dicts[i] = dict_artist
        
            
    encoder = json.JSONEncoder()

    #Preparation of outputs of the function
    array_artists_json = encoder.encode(array_artists_dicts)
        
    return(array_artists_json)

In [29]:
out = main_treatment_HTML_to_JSON("./data")

Opening finished for filelot1.html
Opening finished for filelot5.html
Opening finished for filelot4.html
Opening finished for filelot3.html
Opening finished for filelot2.html
Opening finished for filelot1.html
Opening finished for filelot5.html
Opening finished for filelot4.html
Opening finished for filelot3.html
Opening finished for filelot2.html


In [30]:
sys.stdout.write(out)

[{"artist": "Rembrandt Harmensz. van Rijn", "works": [{"title": "Self-Portrait in a Cap and Scarf with Face dark: Bust", "currency": "USD", "amount": 8125.0}, {"title": "Christ at Emmaus: The smaller Plate", "currency": "USD", "amount": 9212.5}]}, {"artist": "Pablo Picasso", "works": [{"title": "Quatre Femmes nues et T\u00eate sculpt\u00e9e, from: La Suite Vollard", "currency": "USD", "amount": 36000.0}, {"title": "Femme assise, robe bleue", "currency": "USD", "amount": 45047500.0}, {"title": "Quatre Femmes nues et T\u00eate sculpt\u00e9e, from: La Suite Vollard", "currency": "USD", "amount": 25000.0}]}, {"artist": "Louis Marcoussis", "works": [{"title": "Bar du porto", "currency": "USD", "amount": 149000.0}, {"title": "Portrait d'Alice Halicka", "currency": "USD", "amount": 182500.0}, {"title": "La Table", "currency": "USD", "amount": 12328.0}]}, {"artist": "Rembrandt Harmensz. van Rijn", "works": [{"title": "Christ at Emmaus: The smaller Plate", "currency": "USD", "amount": 9212.5}]}